# NLP - Security Reports Analysis

## Appendix:

Click on the links:

[Business Problem](#Business-Problem) <br>
[Imports Libraries](#Imports-Libraries) <br>
[Read Data](#Read-Data) <br>
[Data Cleaning and Aggregation](#Data-Cleaning-and-Aggregation):
- Filling NULL values in [Description] column
- Converting the type of [Description] to string
- Removing Harakat/ Tashkeel
- Normalizing some letters
- Sippliting the text
- Removing Punctuation
- Removing Tatweel 
- Removing repeated chars
- Stemming <br>

[Classification](#Classification)  <br>
[Tokenization](#Tokenization)  <br>

## Business Problem

In this project we provide a proposal to use NLP techniques in analysing "كلنا أمن" App's data. <br> 
It is made as a participation in Absher-thon.

## Imports Libraries

In [2]:
import pandas as pd

In [3]:
from camel_tools.utils.dediac import dediac_ar

In [4]:
from camel_tools.utils.normalize import normalize_alef_ar
from camel_tools.utils.normalize import normalize_teh_marbuta_ar

In [5]:
from camel_tools.tokenizers.word import simple_word_tokenize

In [6]:
from pyarabic.araby import strip_tatweel

In [7]:
from farasa.stemmer import FarasaStemmer

In [8]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

In [9]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

In [10]:
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVC

In [11]:
from sklearn.model_selection import train_test_split

## Read Data

In [28]:
df = pd.read_excel('DatasetApril_classification.xlsx')
df.head(2)

,رقم البلاغ,Name,Mobile,ID,Date,Day,City,Class,Traffic,Type,Currently,Cyber,Platform,State_account,Link,Account,Type_people,Description,State_report,Label
0,R1,فهد ابراهيم احمد الصالحي,557255350,1101611893,2021-01-25,الاثنين,الدمام,الاتجار بالاشخاص,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,العمل الجبري,اخوي يخليني اروح للبقالة غصب,مغلق,Fake
1,R2,فهد ابراهيم احمد الصالحي,557255350,1101611893,2021-01-28,الخميس,الرياض,الاتجار بالاشخاص,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,العمل الجبري,كل يوم اخوي يجبرني اروح للبقالة بداله,مغلق,Fake


In [29]:
df.columns

Index(['رقم البلاغ', 'Name', 'Mobile', 'ID', 'Date', 'Day', 'City', 'Class',
       'Traffic', 'Type', 'Currently', 'Cyber', 'Platform', 'State_account',
       'Link', 'Account', 'Type_people', 'Description', 'State_report',
       'Label'],
      dtype='object')

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 993 entries, 0 to 992
Data columns (total 20 columns):
رقم البلاغ       993 non-null object
Name             993 non-null object
Mobile           993 non-null int64
ID               993 non-null int64
Date             993 non-null datetime64[ns]
Day              993 non-null object
City             993 non-null object
Class            993 non-null object
Traffic          278 non-null object
Type             32 non-null object
Currently        32 non-null object
Cyber            210 non-null object
Platform         210 non-null object
State_account    210 non-null object
Link             0 non-null float64
Account          56 non-null object
Type_people      212 non-null object
Description      960 non-null object
State_report     993 non-null object
Label            993 non-null object
dtypes: datetime64[ns](1), float64(1), int64(2), object(16)
memory usage: 155.3+ KB


## Data Cleaning and Aggregation

Filling NULL values in [Description] column

In [31]:
#df[df.Description != ' None']
#df[df.Description.notna()]
df['Description'] = df['Description'].fillna("لا يوجد")
df.head(2)

,رقم البلاغ,Name,Mobile,ID,Date,Day,City,Class,Traffic,Type,Currently,Cyber,Platform,State_account,Link,Account,Type_people,Description,State_report,Label
0,R1,فهد ابراهيم احمد الصالحي,557255350,1101611893,2021-01-25,الاثنين,الدمام,الاتجار بالاشخاص,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,العمل الجبري,اخوي يخليني اروح للبقالة غصب,مغلق,Fake
1,R2,فهد ابراهيم احمد الصالحي,557255350,1101611893,2021-01-28,الخميس,الرياض,الاتجار بالاشخاص,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,العمل الجبري,كل يوم اخوي يجبرني اروح للبقالة بداله,مغلق,Fake


Converting the type of [Description] to string

In [32]:
df.Description = df.Description.astype(str)

Removing Harakat/ Tashkeel



In [33]:
df.Description = df.Description.apply(dediac_ar)
df['Description'][200]

'السيارة تعطلت في نص طريق صاد'

Normalizing some letters

In [34]:
def ortho_normalize(text):
    text = normalize_alef_ar(text)
    text = normalize_teh_marbuta_ar(text)
    return text

df.Description = df.Description.apply(ortho_normalize)
df['Description'].unique()

array(['اخوي يخليني اروح للبقاله غصب',
       'كل يوم اخوي يجبرني اروح للبقاله بداله',
       'قريبي يجبر السائق على اعمال ليست من اختصاصه',
       'شفت اطفال يبيعون عند الاشاره',
       'جارتي تجبر عاملتها على اعمال اخرى', 'في الحديقه طفل يبيع مناديل',
       'اخوي غاصبني اروح للبقاله',
       'شركه العاملات تجبر العاملات على اعمال متواصله', 'العمل الجبري',
       'المتاجره بالاعضاء', 'اعرف عصابه يشترون اعضاء ويبيعونها ',
       'المتاجره بالتسول', 'وش يعني متاجره بالتسول',
       'دايم اشوف عمال يشحذون عند البقاله',
       'عند المسجد نصابين يشحذون فلوس', 'شفت عند الاشاره حرمه تطلب فلوس',
       'بس ابغى اجرب', 'تجربه', 'مافيه وصف', 'اختي دايم تشحذني فلوس',
       'جوا شحاذين لبيتنا', 'هذا بلاغ مزيف', 'للتجربه فقط',
       'شفت رجل يتنكر بزي امراه ليتسول', 'لا يوجد',
       'شفت عامل البقاله يتسول', 'المتاجره بالجنس', '-', 'للتجربه',
       'الممارسات الشبيهه بالرق', 'اقربائي يعاملون الخدم مثل الرق',
       'جارنا يغصب السواق يوصله',
       'راعي المزرعه يجبر العمال على العمل من الفج

Sippliting the text



In [35]:
df.Description = df.Description.apply(simple_word_tokenize)

Removing Punctuation



In [36]:
def removePunctuation(text):
    return " ".join([word for word in text if word.isalnum()])

df.Description = df.Description.apply(removePunctuation)

In [37]:
df['Description'][200]

'السياره تعطلت في نص طريق صاد'

Removing Tatweel 



In [39]:
def Tatweel(text):
    return strip_tatweel(text)

df.Description = df.Description.apply(Tatweel)

df['Description'].unique()

array(['اخوي يخليني اروح للبقاله غصب',
       'كل يوم اخوي يجبرني اروح للبقاله بداله',
       'قريبي يجبر السائق على اعمال ليست من اختصاصه',
       'شفت اطفال يبيعون عند الاشاره',
       'جارتي تجبر عاملتها على اعمال اخرى', 'في الحديقه طفل يبيع مناديل',
       'اخوي غاصبني اروح للبقاله',
       'شركه العاملات تجبر العاملات على اعمال متواصله', 'العمل الجبري',
       'المتاجره بالاعضاء', 'اعرف عصابه يشترون اعضاء ويبيعونها',
       'المتاجره بالتسول', 'وش يعني متاجره بالتسول',
       'دايم اشوف عمال يشحذون عند البقاله',
       'عند المسجد نصابين يشحذون فلوس', 'شفت عند الاشاره حرمه تطلب فلوس',
       'بس ابغى اجرب', 'تجربه', 'مافيه وصف', 'اختي دايم تشحذني فلوس',
       'جوا شحاذين لبيتنا', 'هذا بلاغ مزيف', 'للتجربه فقط',
       'شفت رجل يتنكر بزي امراه ليتسول', 'لا يوجد',
       'شفت عامل البقاله يتسول', 'المتاجره بالجنس', '', 'للتجربه',
       'الممارسات الشبيهه بالرق', 'اقربائي يعاملون الخدم مثل الرق',
       'جارنا يغصب السواق يوصله',
       'راعي المزرعه يجبر العمال على العمل من الفجر'

Removing repeated chars

In [40]:
def remove_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1', text)

Stemming

In [ ]:
stemmer = FarasaStemmer()
def Stemming2(text):
    return (stemmer.stem(text))

df['Description1'] = df['Description1'].apply(Stemming2)

In [114]:
df.head(2)

,رقم البلاغ,Name,Mobile,ID,Date,Day,City,Class,Traffic,Type,...,Cyber,Platform,State_account,Link,Account,Type_people,Description,State_report,Label,Description1
0,R1,فهد ابراهيم احمد الصالحي,557255350,1101611893,2021-01-25,الاثنين,الدمام,الاتجار بالاشخاص,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,العمل الجبري,اخوي يخليني اروح للبقاله غصب,مغلق,Fake,اخوي يخليني اروح بقاله غصب
1,R2,فهد ابراهيم احمد الصالحي,557255350,1101611893,2021-01-28,الخميس,الرياض,الاتجار بالاشخاص,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,العمل الجبري,كل يوم اخوي يجبرني اروح للبقاله بداله,مغلق,Fake,كل يوم اخوي أجبر اروح بقاله دال


## Classification

Pipeline for text data & feature selection

In [125]:
text_features = 'Description1'

text_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer())
])

Pipeline for categorical data

In [ ]:
categorical_features = ['رقم البلاغ', 'Name', 'Mobile', 'ID', 'Date', 'Day', 'City', 'Class',
       'Traffic', 'Type', 'Currently', 'Cyber', 'Platform', 'State_account',
       'Link', 'Account', 'Type_people', 'State_report']

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='Unknown')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

Combine preprocessing with ColumnTransformer

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('text', text_transformer, text_features),
        ('cat', categorical_transformer, categorical_features)
])

In [129]:
clf_pipe =  Pipeline(steps=[('preprocessor', preprocessor),
                   ("model", SVC())
])

In [130]:
X = df[['رقم البلاغ', 'Name', 'Mobile', 'ID', 'Date', 'Day', 'City', 'Class',
       'Traffic', 'Type', 'Currently', 'Cyber', 'Platform', 'State_account',
       'Link', 'Account', 'Type_people', 'Description1', 'State_report']]

In [131]:
X_train, X_test, y_train, y_test = train_test_split(X, df['Label'], test_size=0.2, random_state=0)

In [132]:
clf_pipe.fit(X_train, y_train)
print("model score: %.3f" % clf_pipe.score(X_test, y_test))

model score: 0.950


In [133]:
df['Prediction'] = clf_pipe.predict(X)

In [134]:
df.head()

,رقم البلاغ,Name,Mobile,ID,Date,Day,City,Class,Traffic,Type,...,Platform,State_account,Link,Account,Type_people,Description,State_report,Label,Description1,Prediction
0,R1,فهد ابراهيم احمد الصالحي,557255350,1101611893,2021-01-25,الاثنين,الدمام,الاتجار بالاشخاص,NaN,NaN,...,NaN,NaN,NaN,NaN,العمل الجبري,اخوي يخليني اروح للبقاله غصب,مغلق,Fake,اخوي يخليني اروح بقاله غصب,Fake
1,R2,فهد ابراهيم احمد الصالحي,557255350,1101611893,2021-01-28,الخميس,الرياض,الاتجار بالاشخاص,NaN,NaN,...,NaN,NaN,NaN,NaN,العمل الجبري,كل يوم اخوي يجبرني اروح للبقاله بداله,مغلق,Fake,كل يوم اخوي أجبر اروح بقاله دال,Fake
2,R3,فهد ابراهيم احمد الصالحي,557255350,1101611893,2021-01-27,الأربعاء,حائل,الاتجار بالاشخاص,NaN,NaN,...,NaN,NaN,NaN,NaN,العمل الجبري,كل يوم اخوي يجبرني اروح للبقاله بداله,مغلق,Fake,كل يوم اخوي أجبر اروح بقاله دال,Fake
3,R4,فهد ابراهيم احمد الصالحي,557255350,1101611893,2021-01-26,الثلاثاء,الدمام,الاتجار بالاشخاص,NaN,NaN,...,NaN,NaN,NaN,NaN,العمل الجبري,كل يوم اخوي يجبرني اروح للبقاله بداله,مغلق,Fake,كل يوم اخوي أجبر اروح بقاله دال,Fake
4,R5,فهد ابراهيم احمد الصالحي,557255350,1101611893,2021-01-25,الاثنين,جدة,الاتجار بالاشخاص,NaN,NaN,...,NaN,NaN,NaN,NaN,العمل الجبري,كل يوم اخوي يجبرني اروح للبقاله بداله,مغلق,Fake,كل يوم اخوي أجبر اروح بقاله دال,Fake


## Tokenization

This step is done to prepare the file to use it in tableau

In [136]:
df.Description1 = df.Description1.apply(simple_word_tokenize)
df.head(2)

,رقم البلاغ,Name,Mobile,ID,Date,Day,City,Class,Traffic,Type,...,Platform,State_account,Link,Account,Type_people,Description,State_report,Label,Description1,Prediction
0,R1,فهد ابراهيم احمد الصالحي,557255350,1101611893,2021-01-25,الاثنين,الدمام,الاتجار بالاشخاص,NaN,NaN,...,NaN,NaN,NaN,NaN,العمل الجبري,اخوي يخليني اروح للبقاله غصب,مغلق,Fake,"[اخوي, يخليني, اروح, بقاله, غصب]",Fake
1,R2,فهد ابراهيم احمد الصالحي,557255350,1101611893,2021-01-28,الخميس,الرياض,الاتجار بالاشخاص,NaN,NaN,...,NaN,NaN,NaN,NaN,العمل الجبري,كل يوم اخوي يجبرني اروح للبقاله بداله,مغلق,Fake,"[كل, يوم, اخوي, أجبر, اروح, بقاله, دال]",Fake


In [137]:
words1 = df.Description1.apply(pd.Series)

In [138]:
new_df = words1.merge(df, left_index = True, right_index = True)

In [139]:
new_df = new_df.melt(id_vars = ['رقم البلاغ', 'Name', 'Mobile', 'ID', 'Date', 'Day', 'City', 'Class',
       'Traffic', 'Type', 'Currently', 'Cyber', 'Platform', 'State_account',
       'Link', 'Account', 'Type_people', 'Description', 'State_report',
       'Label', 'Description1', 'Prediction'] , value_name = "word").drop("variable", axis = 1).dropna(subset=['word'])

In [140]:
new_df

,رقم البلاغ,Name,Mobile,ID,Date,Day,City,Class,Traffic,Type,...,State_account,Link,Account,Type_people,Description,State_report,Label,Description1,Prediction,word
0,R1,فهد ابراهيم احمد الصالحي,557255350,1101611893,2021-01-25,الاثنين,الدمام,الاتجار بالاشخاص,NaN,NaN,...,NaN,NaN,NaN,العمل الجبري,اخوي يخليني اروح للبقاله غصب,مغلق,Fake,"[اخوي, يخليني, اروح, بقاله, غصب]",Fake,اخوي
1,R2,فهد ابراهيم احمد الصالحي,557255350,1101611893,2021-01-28,الخميس,الرياض,الاتجار بالاشخاص,NaN,NaN,...,NaN,NaN,NaN,العمل الجبري,كل يوم اخوي يجبرني اروح للبقاله بداله,مغلق,Fake,"[كل, يوم, اخوي, أجبر, اروح, بقاله, دال]",Fake,كل
2,R3,فهد ابراهيم احمد الصالحي,557255350,1101611893,2021-01-27,الأربعاء,حائل,الاتجار بالاشخاص,NaN,NaN,...,NaN,NaN,NaN,العمل الجبري,كل يوم اخوي يجبرني اروح للبقاله بداله,مغلق,Fake,"[كل, يوم, اخوي, أجبر, اروح, بقاله, دال]",Fake,كل
3,R4,فهد ابراهيم احمد الصالحي,557255350,1101611893,2021-01-26,الثلاثاء,الدمام,الاتجار بالاشخاص,NaN,NaN,...,NaN,NaN,NaN,العمل الجبري,كل يوم اخوي يجبرني اروح للبقاله بداله,مغلق,Fake,"[كل, يوم, اخوي, أجبر, اروح, بقاله, دال]",Fake,كل
4,R5,فهد ابراهيم احمد الصالحي,557255350,1101611893,2021-01-25,الاثنين,جدة,الاتجار بالاشخاص,NaN,NaN,...,NaN,NaN,NaN,العمل الجبري,كل يوم اخوي يجبرني اروح للبقاله بداله,مغلق,Fake,"[كل, يوم, اخوي, أجبر, اروح, بقاله, دال]",Fake,كل
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11241,R319,فيصل عبدالرحمن بن عبدالعزيز الدوسري,505915575,1101451852,2021-01-25,الاثنين,حائل,المرور,تفحيط او تجمهر,NaN,...,NaN,NaN,NaN,NaN,في شارع باء كل نهايه اسبوع يتجمعون ناس والتفحي...,مغلق,Real,"[في, شارع, باء, كل, نهاي, أسبوع, تجمع, ناس, تف...",Real,حي
11855,R933,ليان احمد محمد البلوي,502234008,1149098079,2021-01-26,الثلاثاء,القصيم,المرور,تفحيط او تجمهر,NaN,...,NaN,NaN,NaN,NaN,في شارع باء كل نهايه اسبوع يتجمعون شباب والتفح...,مفتوح,Real,"[في, شارع, باء, كل, نهاي, أسبوع, تجمع, شاب, تف...",Real,حي
11857,R935,ليان احمد محمد البلوي,502234008,1149098079,2021-04-26,الاثنين,المدينة المنورة,المرور,تفحيط او تجمهر,NaN,...,NaN,NaN,NaN,NaN,في شارع باء كل نهايه اسبوع يتجمعون ناس والتفحي...,مفتوح,Real,"[في, شارع, باء, كل, نهاي, أسبوع, تجمع, ناس, تف...",Real,حي
11858,R936,ليان احمد محمد البلوي,502234008,1149098079,2021-01-26,الثلاثاء,جدة,المرور,تفحيط او تجمهر,NaN,...,NaN,NaN,NaN,NaN,في شارع باء كل نهايه اسبوع يتجمعون ناس والتفحي...,مفتوح,Real,"[في, شارع, باء, كل, نهاي, أسبوع, تجمع, ناس, تف...",Real,حي
